<a href="https://colab.research.google.com/github/Lucia-KIM/Paper-Review-and-Practice/blob/main/transformer3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
import torch

torch.__version__

'1.7.1+cu101'

## 트랜스포머 아키텍쳐 구현

### Multi Head Attention 

- Scaled Dot-Product Attention 구현해서, 
- Multi-Head Attention으로
- Scaled Dot-Product attention을 겹쳐놓은 h개의 Scaled Dot-Product attention(Multi-Head attention)을 사용
- 각각의 Attention에는 Query, Key, Value가 사용되었는데, 이 세 값들을 바로 사용하는 것이 아니라 h개의 Attention각각에 대해서 다르게 초기화된 Parameter Matrix를 곱하여 (Projection) 사용한다. 
- 즉, 이 과정에서 h개의 다른 값들을 얻게되고 이를 concat해서 attention결과물을 얻기 때문에 하나의 Scaled Dot-Product Attention을 사용할 때보다 주어진 Query, Key, Value에 대해서 보다 다양한 상황(subspace)에 대한 attention을 계산할 수 있게 된다. 

#### 1) Multi-Head Attention은 쿼리, 키, 벨류가 Linear로 각각 scaled dot-product attention에 들어가서 concat되는 형식임 
#### 2) Scaled Dot-Product Attention은 Q와 K를 행렬 곱하고, 스케일 한 다음에 마스크를 씌우고(선택) 소프트 맥스를 거친 결과를 V와 행렬곱한다. 
- Query가 들어오게 되면 Key값과의 계산을 통하여 기준 값 Key에 대한 Query의 상대적인 weight(softmax)를 계산한다.
- 이 Weight를 Value에 곱함으로써 Query와 Key의 연관성을 기준으로 한 새로운 값을 얻게 된다.



In [83]:
import torch.nn as nn
# TORCH.NN: 신경망(neural network) 구조가 디자인 된 모듈과 클래스들을 제공, 필요에 따라  커스터마이즈하여 사용
# 하이퍼 파라미터로 hidden_dim, n_heads, dropout_ratio 지정 

class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, dropout_ratio, device): 
        super().__init__()   # 생성자 

        assert hidden_dim % n_heads == 0  
        # assert는 뒤의 조건이 True가 아니면 AssertError를 발생
        # 나머지가 0인 경우를 찾는다. 

        self.hidden_dim = hidden_dim  # 하나의 단어에 대한 임베딩 차원
        self.n_heads = n_heads # head의 개수 = scaled dot-product attention의 개수
        self.head_dim = hidden_dim // n_heads # 임베딩 차원을 헤드 개수로 나눈 몫, 즉 각 헤드에서의 임베딩 차원

        self.fc_q = nn.Linear(hidden_dim, hidden_dim) # Query 값에 적용될 fc레이어
        self.fc_k = nn.Linear(hidden_dim, hidden_dim) # Key값에 적용될 fc레이어
        self.fc_v = nn.Linear(hidden_dim, hidden_dim) # Value 값에 적용될 fc레이어

        self.fc_o = nn.Linear(hidden_dim, hidden_dim) # output linear
        self.dropout = nn.Dropout(dropout_ratio) # 드롭 아웃 비율
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device) 
        # 이후 스케일로 나눈 값에 소프트맥스를 씌워서 사용할 예정 

    def forward(self, query, key, value, mask = None):
        # mask = None인 경우

        batch_size = query.shape[0] # 쿼리 행렬의 행의 개수
        # query의 모양은 [batch_size, query_len, hidden_dim] 형태, 여기서 인덱스 0번 batch_size을 받음

        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)

        # hidden_dim -> n_heads * head_dim 형태로 변경
        # --> 하나의 단어에 대한 임베딩 차원을 각 헤드의 임베딩 차원*어텐션의 갯수로 변경
        # n_heads(h)개의 서로 다른 어텐션 컨셉을 학습하게 함

        # view()는 tensor의 모양을 바꾸는데 사용(데이터의 구조가 변경될 뿐 순서는 변경되지 않는다)
        # view가 반환한 tensor는 원본 tensor와 기반이 되는 data를 공유한다. 만약 반환된 tensor의 값이 변경된다면, viewed되는 tensor에서 해당하는 값이 변경된다.
        # permute(순서 인덱스)는 모든 차원들을 맞교환할 수 있다.
        # 지금까지 Q는 [batch_size, query_len, hidden_dim] 형태로 저장되었음. 
        # 이걸 view()를 사용해 Q: [batch_size, n_heads, query_len, head_dim]로 텐서의 모양을 바꿔줌 
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0,2,1,3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0,2,1,3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0,2,1,3)

        
        # (Attention Energy) Scaled Dot-Product Attention을 만들기 위해, 
        # input은 d_k dimension의 queries와 keys, 그리고 d_v dimension의 values로 구성된다. 
        # query와 key의 dot product를 계산하여 \sqrt{d_k}로 나눈다.
        # \sqrt{d_k}로 나누어 준, 다시말해 scaled하였기 때문에 Scaled Dot-Product Attention
        # 이후 softmax function**을 사용하여 values에 대한 weights를 얻어 낸다.
        energy = torch.matmul(Q, K.permute(0,1,3,2))/self.scale
        # matmul(): 3차원 이상의 행렬끼리 곱
        # energy: [batch_size, n_heads, query_len, key_len]
        # 여기서 Q는 [batch_size, n_heads, query_len, head_dim],
        # K.permute(0,1,3,2)는 [batch_size, n_heads, head_dim, key_len]
        # 그럼 head_dim* query_len = query_len이고, head_dim*key_len = key_len인가? 


        # 마스크를 사용하는 경우
        if mask is not None:
            # 마스크를 부분을 아주 작은 값으로 채우기 -1e10
            energy = energy.masked_fill(mask==0, -1e10)

        # 소프트맥스로 어텐션 스코어 계산 : 각 단어에 대한 확률
        attention = torch.softmax(energy, dim=-1)
        # query와 key에 대한 dot-product를 계산하면 각각의 query와 key 사이의 유사도를 구할 수 있게 된다.
        # attention: [batch_size, n_heads, query_len, key_len]

        # 여기에서 Scaled Dot-Product Attention을 계산
        x = torch.matmul(self.dropout(attention), V)
        # key와 value는 attention이 이루어지는 위치에 상관없이 같은 값을 갖게 되는데, 
        # 여기서 x는 [batch_size, n_heads, query_len, head_dim] 이다.

        x = x.permute(0, 2, 1, 3).contiguous()
        # x: [batch_size, query_len, n_heads, head_dim]
        # contiguous()는 새로운 메모리에 할당하여 주소값 재배열이 가능하다. 
        
        x = x.view(batch_size, -1, self.hidden_dim)
        #self.head_dim = hidden_dim // n_heads 였으니, head_dim*n_heads = hidden_dim
        # 그래서 x는 [batch_size, query_len, hidden_dim]

        x = self.fc_o(x)
        # x: [batch_size, query_len, hidden_dim]

        return x, attention

## Position-wise Feedforward Networks
- 인코더와 디코더의 각각 position(개별 단어마다)에 개별적으로 동일하게 적용되는 fully connected feed-forward network sub-layerrk 있음 
- ReLU 함수를 포함한 두개의 선형 변환(linear transformation 1, 2)으로 구성됨
- input(x) -> linear transformation_1 -> ReLU -> linear transformation_2 

In [84]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hidden_dim, pf_dim, dropout_ratio):
        super().__init__()

        self.fc_1 = nn.Linear(hidden_dim, pf_dim) # linear transformation_1
        self.fc_2 = nn.Linear(pf_dim, hidden_dim) # linear transformation_2
        # hidden_dim: 하나의 단어에 대한 임베딩 차원
        # pf_dim: Feedforward 레이어에서의 내부 임베딩 차원
        # hidden_dim 만큼의 차원이 들어와서 최종 hidden_dim 만큼의 차원을 내보냄
        # 다시말해, 입력과 출력의 차원이 동일함

        self.dropout = nn.Dropout(dropout_ratio) # dropout_ratio: 드롭아웃(dropout) 비율

    def forward(self, x):
        x = self.dropout(torch.relu(self.fc_1(x)))
        # 선형변화 1에 ReLU를 적용한 x
        # x: [batch_size, seq_len, hidden_dim]

        x = self.fc_2(x)
        # 이어서 선형변화 2를 적용함 

        return x

## Encoder Layer
- 하나의 인코더 레이어를 구현
- 입력과 출력의 차원이 같다.
- 트랜스포머는 이런 인코더 레이어를 여러번 중첩하여 사용한다. 

### 간단한 구조는, 
    1) 소스에 대한 어텐션 + 정규화
    2) positionwise feedforward + 정규화

In [85]:
class EncoderLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio, device):
        super().__init__()

        self.self_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.ff_layer_norm = nn.LayerNorm(hidden_dim)
        # Layer Normalization
        # 정규화란 무언가를 표준화 시키거나 다른 것과 비교하기 쉽도록 바꾸는 것
        # 따라서, 정규화는 데이터의 범주를 바꾸는 작업으로 스케일이 곧 정규화
        #선형대수학에서 놈은 벡터의 크기(magnitude) 또는 길이(length)를 측정하는 방법을 의미

        self.self_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hidden_dim, pf_dim, dropout_ratio)
        self.dropout = nn.Dropout(dropout_ratio)

    # 하나의 임베딩이 복제되어 쿼리, 키, 벨류로 입력되는 방식
    def forward(self, src, src_mask):
        # src(소스는) [batch_size, src_len, hidden_dim]
        # src_mask : [batch_size, src_len]

        # 인코더 self-attention layer
        _src, _ = self.self_attention(src, src, src, src_mask)
        # 필요한 경우 self attention에서 마크스 행렬을 이용하여 어텐션할 단어를 조절 가능
        # src-src 이거나, src-src_masked
        # 디코더에서는 인코더와 달리 순차적으로 결과를 만들어야 하기 때문에 mask를 사용함
        # 다시말해 특정 단어 position i보다 뒤에 있는 position에 어텐션을 주지 못하게 함

        src = self.self_attn_layer_norm(src + self.dropout(_src))
        # dropout, residual connection and layer norm
        # 각각의 계산을 진행할 때는 Residual Connection{=LayerNorm(x + Sublayer(x))}과 Layer normalization을 적용한다.
        # Residual Connection의 개념은 기존에 학습한 정보를 보존하고, 거기에 추가적으로 학습한 정보를 더하는 형식이다.
        # x의 아웃풋이 y라고 할때, y=f(x)는 direct로 학습한 경우라고 생각할 수 있다. 아웃풋인 y는 x를 통해 새롭게 학습한 정보를 의미한다.
        # y=f(x)인 경우는 기존에 학습한 정보를 보존하지 않고 변형시켜 새롭게 생성하는 정보이다.
        # 이 경우 레이어의 깊이 깊어질수록 한번에 학습해야 할 mapping이 너무 많아져 학습이 어려워지는 문제가 발생한다.
        # 반면에 Residual Connection은 y=f(x)+x로 이전에 학습한 내용 x를 더해주어(보존)하여 추가되는 학습만 진행하면 된다는 장점이 있다.
        # 논문에서는 Residual Connection이 포함된 self attention의 장점으로 layer당 계산량이 줄어든다고 기재해 놓았다. 
        
        # position-wise feedforward
        _src = self.positionwise_feedforward(src)

        src = self.ff_layer_norm(src + self.dropout(_src))
        # dropout, residual and layer norm
        # src: [batch_size, src_len, hidden_dim]
        
        return src

## 인코더 아키텍처

: 전체 인코더 아키텍처 정의

- 전처리 후 토큰화 된 소스를 input embedding
- positional encoding을 거쳐 encoder 박스로 들어감.
    * 트랜스포머는 단어의 순서(sequence)를 이용하기 위해 position에 대한 정보를 추가해 주어야 함.
    * 논문에서는 서로 다른 빈도의 사인과 코사인 함수를 이용한다. 


In [86]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio, device, max_length=100):
        super().__init__()

        self.device = device
        
        # nn.Embedding()는 임베딩 층(embedding layer)을 만들어 훈련 데이터로부터 처음부터 임베딩 벡터를 학습하는 방법
        # nn.Embedding()을 사용하여 학습가능한 임베딩 테이블 만든다. 
        # nn.Embedding은 크게 두 가지 인자를 받는데, 
        # 1) num_embeddings : 임베딩을 할 단어들의 개수. 다시 말해 단어 집합의 크기
        # 2) embedding_dim : 임베딩 할 벡터의 차원
        self.tok_embedding = nn.Embedding(input_dim, hidden_dim)
        # input_dim: 하나의 단어에 대한 원 핫 인코딩 차원
        # hidden_dim: 하나의 단어에 대한 임베딩 차원

        # 인풋 차원으로 들어온 것을 임베딩 차원으로 바꿔주고, 
        # 위치에 대한 정보 값을 더해주기 위해 pos 임베딩 레이어를 추가한다. 

        self.pos_embedding = nn.Embedding(max_length, hidden_dim)
        # max_length: 문장 내 최대 단어 개수

        self.layers = nn.ModuleList([EncoderLayer(hidden_dim, n_heads, pf_dim, dropout_ratio, device) for _ in range(n_layers)])
        # layer의 갯수는 앞서 만들어 놓은 인코더 레이어의 수를 가져온다.
        # nn.ModuleList()는 각 레이어를 리스트에 전달하고 레이어의 iterator(반복)를 만든다.
        # 인코더 레이어의 수 만큼 for를 돌려서 layer의 수를 정한다.

        self.dropout = nn.Dropout(dropout_ratio)
        self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

    def forward(self, src, src_mask):

        batch_size = src.shape[0]  # 문장의 개수
        src_len = src.shape[1] # 각 문장들 중 단어의 개수가 가장 많은 문장의 단어 개수
        # src는 [batch_size, src_len]로 구성되어 있음

        # positional encoder를 논문과 다르게 학습하는 형태로 구현
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        # unsqueeze함수는 지정한 자리에 1인 차원을 생성하는 함수이다. 
        # repeat(object, n)는 object를 n번 반복함 - 각각의 문장마다 수행하도록 repeat함
        # 여기서는 소스의 길이만큼 1차원 텐서를 만들어서, unsqueeze()를 통해 2차원으로 만들고,
        # unsqueeze로 만든 1차원 자리에 배치 사이즈만큼의 차원으로 바꿔준다. 
        # 결국 pos는 [batch_size, src_len] 형태가 된다.

        # 소스 문장의 임베딩과 위치 임베딩을 더한 것을 실제 입력 값으로 사용
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))

        # 모든 인코더 레이어를 차례대로 거치면서 순전파(forward) 수행
        for layer in self.layers:
            src = layer(src, src_mask)
        # src: [batch_size, src_len, hidden_dim]

        return src # 마지막 레이어의 출력을 반환

## Decoder Layer 
: 하나의 디코더 레이어 정의
- 트랜스포머의 디코더는 디코더 레이어를 여러 번 중첩해 사용
- 디코더 레이어에서는 2개의 Multi-Head Attention 레이어가 사용 됨
    * 타겟 문장에서 각 단어는 다음 단어가 무엇인지 알 수 없도록 만들기 위해 Masked Multi-Head Attention 사용
    * 인코더에서 출력된 내용에 대한 Multi-Head Attention
- 이후에 positionwise feedforward를 진행함

### 전체 구조를 간단히 보면,
    1) 타겟에 대한 마스크 어텐션 + 정규화
    2) 인코더 출력에 대한 어텐션 + 정규화
    3) positionwise feedforward + 정규화

In [87]:
class DecoderLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio, device):
        super().__init__()

        self.self_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.ff_layer_norm = nn.LayerNorm(hidden_dim)
        self.self_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
        self.encoder_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hidden_dim, pf_dim, dropout_ratio)
        self.dropout = nn.Dropout(dropout_ratio)

    # 인코더의 출력 값(enc_src)을 어텐션(attention)하는 구조 
    def forward(self, trg, enc_src, trg_mask, src_mask):
        # trg: [batch_size, trg_len, hidden_dim]
        # enc_src: [batch_size, src_len, hidden_dim]
        # trg_mask: [batch_size, trg_len]
        # src_mask: [batch_size, src_len]

        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)
        # 타겟 데이터 스스로 자신에게 어텐션하는 구조
        # 쿼리, 키, 벨류 모두 자기 자신을 넣을 수 있게 trg로 

        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
        # dropout, residual connection and layer norm

        # 디코더의 쿼리(Query)를 이용해 인코더를 어텐션(attention)
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)
        # encoder_attention은 x(단어)와 attention(확률)을 반환 
        # 쿼리는 디코더에 포함되어 있는 출력 단어들에 대한 정보 (trg)
        # 인코더에서 가장 마지막으로 출력된 값(enc_src)을 키로 사용한다.
        
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
        # encoder에서 받은 x(_trg)와 trg로 encoder attention layer의 normalization 한다.
        # dropout, residual connection and layer norm
        # trg: [batch_size, trg_len, hidden_dim]

        # 타겟을 masked 어텐션한 것 1층
        # 인코더의 출력 내용 어텐션 1층
        # 이제 positionwise feedforward 할 차례임
        _trg = self.positionwise_feedforward(trg)
        trg = self.ff_layer_norm(trg + self.dropout(_trg))

        return trg, attention
        # trg: [batch_size, trg_len, hidden_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

## 디코더 아키텍처
: 전체 디코더 아키텍처 정의
- 이번에도 논문과 다르게, 위치 임베딩(positional embedding)을 학습하는 형태로 구현

In [88]:
class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio, device, max_length=100):
        super().__init__()

        self.device = device
        self.tok_embedding = nn.Embedding(output_dim, hidden_dim)
        # output_dim: 하나의 단어에 대한 원 핫 인코딩 차원
        # hidden_dim: 하나의 단어에 대한 임베딩 차원
        # 단어의 개수와 같은 차원을 임베딩 차원으로 바꿔주고, 
        
        self.pos_embedding = nn.Embedding(max_length, hidden_dim)
        # 위치에 대한 정보를 주기위해 전체 시퀀스의 길에 해당하는 차원을 hidden_dim으로 바꿔준다.

        self.layers = nn.ModuleList([DecoderLayer(hidden_dim, n_heads, pf_dim, dropout_ratio, device) for _ in range(n_layers)])
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_ratio)
        self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

    # 인코더의 마지막 레이어에서 나온 출력값과 타켓문장에 대한 정보를 받는다.
    def forward(self, trg, enc_src, trg_mask, src_mask):
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        # trg: [batch_size, trg_len]
        
        # 출력 문장도 0부터 단어의 개수에 대한 위치 정보를 담기 위해 초기화한 텐서를 생성하여 각 문장에 대해 동일하게 적용할 수 있게 만든다. 
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        # pos: [batch_size, trg_len]

        # 문장의 임베딩 값에 위치에 대한 정보를 더한 값을 사용
        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))
        # 토큰 임베딩을 스케일해서, positional 임베딩과 더함. 
        # 이걸 드롭아웃 처리
        # trg에 hidden_dim이 더해져서 [batch_size, trg_len, hidden_dim] 구조가 됨

        # self.layers는 nn.ModuleList()로 decoder layers의 레이어를 하나씩 담은 리스트
        for layer in self.layers:
            # 소스 마스크와 타겟 마스크 모두 사용
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)            
        # trg: [batch_size, trg_len, hidden_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        # 디코더 레이어를 여러번 거쳐 마지막에 나온 trg 값에 출력을 위한 linear layer을 거치게 만든다.
        output = self.fc_out(trg) # fc_out은 output을 위한 linear layer
        # self.fc_out가 nn.Linear(hidden_dim, output_dim)이니
        # 최종 output은 [batch_size, trg_len, output_dim] 형태
        
        return output, attention

## 트랜스포머(Transformer) 아키텍처
: 최종 전체 트랜스포머 모델 정의

In [89]:
class Transformer(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, trg_pad_idx, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        # 소스 문장의 <pad> 토큰(padding)에 대하여 마스크(mask) 값을 0으로 설정

    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        # src는 [batch_size, src_len]의 형태, 패딩 토큰을 제외하고 
        # 일반 소스에는 unsqueeze() 함수로 [batch_size, 1, 1, src_len] 구조로 바꿔준다.
        # src_mask: [batch_size, 1, 1, src_len]

        return src_mask

    # 타겟 문장에서는 다음 단어가 무엇인지 모르게 mask사용
    def make_trg_mask(self, trg):
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        # trg: [batch_size, trg_len]
        # trg_pad_mask: [batch_size, 1, 1, trg_len]

        trg_len = trg.shape[1]
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()
        # tril()은 행렬의 대각행렬 아래쪽을 지정한 수로 바꾸고 위쪽은 0으로 반환
        # 여기서는 ones()를 통해 1로 바꿈, 아래와 같은 형식
        """ (마스크 예시)
        1 0 0 0 0
        1 1 0 0 0
        1 1 1 0 0
        1 1 1 1 0
        1 1 1 1 1
        """
        # trg_sub_mask: [trg_len, trg_len]

        trg_mask = trg_pad_mask & trg_sub_mask
        # &는 비트연산자로 AND 연산한다. 둘다 참일때만 만족
        # 여기서는 Element wise로 and 연산을 수행한다. 즉, 각 행렬의 원소끼리만 곱한 것을 의미한다.
        # 결과적으로는 두 마스크에서 값이 둘다 1인 경우에만 어텐션 스코어를 만들 수 있도록 만든다.

        # trg_mask: [batch_size, 1, trg_len, trg_len]
        return trg_mask

    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        # src: [batch_size, src_len]
        # trg: [batch_size, trg_len]
        # src_mask: [batch_size, 1, 1, src_len]
        # trg_mask: [batch_size, 1, trg_len, trg_len]

        # 인코더에 소스 문장을 넣어 인코더 출력값을 생성
        enc_src = self.encoder(src, src_mask)
        # enc_src: [batch_size, src_len, hidden_dim]

        # 디코더는 인코더의 출력값(enc_src)에 매번 어텐션할 수 있도록 만든다.
        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)
        # output: [batch_size, trg_len, output_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        return output, attention
        # 최종 출력된 output값이 번역 결과가 된다. 

## 모델 학습하기 

### 작업 순서 
    1) 학습할 데이터 불러오기
    2) 데이터 전처리
    3) 모델 초기화 및 파라미터 설정
    4) 모델 학습 및 검증 (loss 계산)
    5) 학습동안 베스트 파라미터 도출 
    6) 학습된 모델 저장  

In [90]:
!pip install torchtext==0.6.0

## 데이터 전처리

In [91]:
%%capture   
#출력 내용이 나오지 않도록 억제

# 토큰화를 지원하는 spacy라이브러리를 사용할 예정 
!python -m spacy download en
!python -m spacy download de

# ! 는 작업의 강제성을 부여, 관리자 권한을 줄때(sudo처럼)


In [92]:
import spacy
spacy_en = spacy.load('en')  # 영어 토큰화 기법이 담긴 객체 생성
spacy_de = spacy.load('de') # 독일어 토큰화

In [93]:
# 각 언어의 토큰화 함수 정의

def tokenize_de(text):
    return [token.text for token in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [token.text for token in spacy_en.tokenizer(text)]


In [94]:
# torchtext.data에는 필드(Field)라는 도구를 제공하는데, 필드를 통해 앞으로 어떤 전처리를 할 것인지를 정의한다.
# 번역 목표 : 소스(SRC) - 독일어, 타겟(TRG) - 영어

from torchtext.data import Field, BucketIterator

SRC = Field(tokenize=tokenize_de, init_token='<sos>', eos_token='<eos>', lower=True, batch_first=True)
TRG = Field(tokenize=tokenize_en, init_token='<sos>', eos_token='<eos>', lower=True, batch_first=True)
# 초기 입력 토큰은 문자열-시작을 알리는 (start-of-string) <SOS> 토큰으로 지정
# 동일하기 마지막 토큰을 알리는 <eos> (end-of-string) 지정


In [95]:
# 파이토치의 약 3만개의 영-독 데이터 셋을 불러온다.
from torchtext.datasets import Multi30k

train_dataset, valid_dataset, test_dataset = Multi30k.splits(exts=(".de",".en"), fields=(SRC, TRG))


In [96]:
# 각 학습, 검증, 테스트 데이터로 분리된 데이터의 양 확인
print(len(train_dataset.examples), len(valid_dataset.examples), len(test_dataset.examples))

print(type(train_dataset.examples[30])) #인덱스 30번의 문장을 불러옴
vars(train_dataset.examples[30])
# vars([object])는 개체의 속성을 리턴해주는 함수
# 다시말하면, torchtext.data.example은 Example 클래스 안에 소스 단어들을 속성으로 가지고 있다고 보면 됨.


29000 1014 1000
<class 'torchtext.data.example.Example'>


{'src': ['ein',
  'mann',
  ',',
  'der',
  'mit',
  'einer',
  'tasse',
  'kaffee',
  'an',
  'einem',
  'urinal',
  'steht',
  '.'],
 'trg': ['a',
  'man',
  'standing',
  'at',
  'a',
  'urinal',
  'with',
  'a',
  'coffee',
  'cup',
  '.']}

In [97]:
#field의 build_vocab() 도구를 사용하면 단어 집합(사전)을 생성할 수 있음
SRC.build_vocab(train_dataset, min_freq=2)
TRG.build_vocab(train_dataset, min_freq=2)
# min_freq : 단어 집합에 추가 시 단어의 최소 등장 빈도 조건을 추가
# 여기서는 최소 두번 이상 나온 단어만 사전으로 생성

print('SRC 단어 집합의 크기 : {}'.format(len(SRC.vocab)))
print('TRG 단어 집합의 크기 : {}'.format(len(TRG.vocab)))

# 생성된 단어 집합 내의 단어들은 .stoi를 통해서 확인 가능
# stoi --> string to i는 해당 단어가 존재하면 인덱스를 반환
print(TRG.vocab.stoi['his'])
# dict 형태로 저장되어 있기 때문에 키 값을 넣어 해당 단어의 인덱스 값을 찾을 수 있음

test = {k:v for k,v in TRG.vocab.stoi.items()}
test.get('mother')  # 496는 인덱스
    

SRC 단어 집합의 크기 : 7855
TRG 단어 집합의 크기 : 5893
27


496

In [98]:
# 문장의 안의 단어의 순서를 유지하여 입력되어야 함
# 이를 위해서는 각 배치에 포함되는 단어의 갯수를 맞춰주면 좋은데, BucketIterator를 사용 함
# BucketIterator는 비슷한 길이를 갖는 데이터를 함께 묶는(batch) Iterator를 정의함. 
# 매 새로운 epoch에서 랜덤한 batch를 생성하는 과정에서 padding을 최소화하기 위해 사용

# batch size란 sample데이터 중 한번에 네트워크에 넘겨주는 데이터의 수를 말한다.
# 가중치와 편향을 수정하는 간격이라고도 함
# 배치 사이즈는 GPU RAM에 맞는 크기로 지정하는 것이 좋음
# 여기서는 논문과 동일하게 128로 하겠음

import torch
from torchtext.data import BucketIterator

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# gpu 사용 가능할시 gpu 사용하게끔 cuda로 올려줌

batch_size = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_dataset, valid_dataset, test_dataset),
    batch_size=batch_size, device=device
)

In [99]:
for i, batch in enumerate(train_iterator):
    src = batch.src
    trg = batch.trg

    print('첫번째 배치 크기 : ', {src.shape})

    # 배치 1번에 포함된 문장 정보 출력
    for i in range(src.shape[1]):
        print(f"인덱스 {i}: {src[0][i].item()}") # 여기에서는 [Seq_num, Seq_len]
    
    # 첫번째 배치만 출력되고 중단
    break

    # 2 는 <sos>, 3은 <eos>, 1은 패딩 토큰을 의미
    # 다시말하면 인덱스 14의 3 <eos> 뒤에는 모두 패딩 토큰으로 이 문장의 길이는 13이다. 

첫번째 배치 크기 :  {torch.Size([128, 29])}
인덱스 0: 2
인덱스 1: 8
인덱스 2: 5792
인덱스 3: 402
인덱스 4: 12
인덱스 5: 14
인덱스 6: 1420
인덱스 7: 27
인덱스 8: 15
인덱스 9: 423
인덱스 10: 12
인덱스 11: 4
인덱스 12: 3
인덱스 13: 1
인덱스 14: 1
인덱스 15: 1
인덱스 16: 1
인덱스 17: 1
인덱스 18: 1
인덱스 19: 1
인덱스 20: 1
인덱스 21: 1
인덱스 22: 1
인덱스 23: 1
인덱스 24: 1
인덱스 25: 1
인덱스 26: 1
인덱스 27: 1
인덱스 28: 1


## 하이퍼 파라미터 설정 및 모델 초기화


In [100]:
INPUT_DIM = len(SRC.vocab) # 소스 언어에 포함되어 있는 언어의 개수
OUTPUT_DIM = len(TRG.vocab)
HIDDEN_DIM = 256
# 논문보다 적은 수의 레이어를 사용함 
# ==인코더/디코더 layer===================================
ENC_LAYERS = 3
DEC_LAYERS = 3
# ===인코더/디코더 헤드==================================
ENC_HEADS = 8
DEC_HEADS = 8
# ==인코더/디코더 feedforward 차원===================================
ENC_PF_DIM = 512
DEC_PF_DIM = 512
# ==인코더/디코더 드롭아웃 비율===================================
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

In [101]:
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
# 생성된 단어 집합 내의 단어들은 .stoi를 통해서 확인 가능
# stoi는 문자를 정수로 바꿔주는 함수 

# 인코더(encoder)와 디코더(decoder) 객체 선언
enc = Encoder(INPUT_DIM, HIDDEN_DIM, ENC_LAYERS, ENC_HEADS, ENC_PF_DIM, ENC_DROPOUT, device)
dec = Decoder(OUTPUT_DIM, HIDDEN_DIM, DEC_LAYERS, DEC_HEADS, DEC_PF_DIM, DEC_DROPOUT, device)

# 모델 트랜스포머로 설정 
model = Transformer(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

- 모델 가중치 파라미터 초기화

In [102]:
# 모델의 파라미터 수를 확인한다.

def count_parametrs(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad) 
    # numel()은 input텐서의 총 요소 수를 반환한다. 
    # 텐서를 생성하고 requires_grad 속성을 True 로 설정하면, 그 tensor에서 이뤄진 모든 연산들을 추적(track)하기 시작한다. 
print(f'The model has {count_parametrs(model):,} trainable parameters')

The model has 9,038,853 trainable parameters


In [103]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)
    # hasattr(object, name)는 object의 속성(attribute) 존재를 확인
    # 만약 argument로 넘겨준 object 에 name 의 속성이 존재하면 True, 아니면 False를 반환
    # m이 weight 속성을 가지고 있고 그 weight의 차원이 1보다 큰 경우,
    # 균일 분포(uniform distribution)로 가중치를 초기화 한다. 

model.apply(initialize_weights)

Transformer(
  (encoder): Encoder(
    (tok_embedding): Embedding(7855, 256)
    (pos_embedding): Embedding(100, 256)
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (ff_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (self_attention): MultiHeadAttentionLayer(
          (fc_q): Linear(in_features=256, out_features=256, bias=True)
          (fc_k): Linear(in_features=256, out_features=256, bias=True)
          (fc_v): Linear(in_features=256, out_features=256, bias=True)
          (fc_o): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (positionwise_feedforward): PositionwiseFeedforwardLayer(
          (fc_1): Linear(in_features=256, out_features=512, bias=True)
          (fc_2): Linear(in_features=512, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
 

- 학습 및 평가 함수 정의

In [104]:
import torch.optim as optim

# Adam optimizer로 학습 최적화
LEARNING_RATE = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

criterion = nn.CrossEntropyLoss(ignore_index= TRG_PAD_IDX)
# 패딩(padding)에 대해서는 값 무시
# orch.nn.CrossEntropyLoss는 nn.LogSoftmax와 nn.NLLLoss의 연산의 조합
# nn.LogSoftmax는 신경망 말단의 결과 값들을 확률개념으로 해석하기 위한 Softmax 함수의 결과에 log 값을 취한 연산이고, 
# nn.NLLLoss는 cross-entropy 손실을 구하는 함수이다.
# 만일 nn.NLLLoss만 쓴 경우에는 모델 마지막 레이어에 Softmax를 사용하게 된다.
# 다시말하면, CrossEntropyLoss는 SoftMax를 적용하고 손실 값을 구하는 함수있다. 

In [105]:
# 모델 학습(train) 함수

def train(model, iterator, optimizer, criterion, clip):
    model.train() # 학습 모드
    epoch_loss = 0

    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()

        output, _ = model(src, trg[:,:-1])
        # 출력 단어의 마지막 인덱스(<eos>)는 제외
        # 입력을 할 때는 <sos>부터 시작하도록 처리
        # 트랜스포머 모델은 output과 attention을 리턴함
        # output: [배치 크기, trg_len - 1, output_dim]
        # trg: [배치 크기, trg_len]

        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        # contiguous()로 새로운 메모리 공간에 데이터를 복사하여 주소값 연속성을 가변적이게 만들어 주고,
        # view() 로 텐서의 모양을 조절한다.
        
        trg = trg[:,1:].contiguous().view(-1)
        # 출력 단어의 인덱스 0(<sos>)은 제외

        # output: [배치 크기 * trg_len - 1, output_dim]
        # trg: [배치 크기 * trg len - 1]

        loss = criterion(output, trg)
        # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
        loss.backward() # 기울기(gradient) 계산
        # forward 함수는 입력 Tensor로부터 출력 Tensor를 계산합니다. backward 함수는 어떤 스칼라 값에 대한 출력 Tensor의 변화도를 전달받고, 동일한 스칼라 값에 대한 입력 Tensor의 변화도를 계산한다.
        # 역전파 단계: 모델의 학습 가능한 모든 매개변수에 대해 손실의 변화도를 계산한다.
        # 내부적으로 각 Module의 매개변수는 requires_grad=True 일 때, 
        # Tensor 내에 저장되므로, 이 호출은 모든 모델의 모든 학습 가능한 매개변수의 변화도를 계산하게 된다.

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        # 매 time-step마다 파라미터에 기울기가 더해지므로, 출력의 길이에 따라 기울기의 크기가 달라진다. 
        # 즉, 길이가 길수록 자칫 기울기가 너무 커질 수 있으므로, 학습률을 조절하여 경사하강법의 업데이트 속도를 조절해야 한다.
        # 너무 큰 학습률을 사용하면 (gradient의 크기인 norm이 너무 큰 경우) 
        # 경사하강법에서 한 번의 업데이트 스텝의 크기가 너무 커져, 자칫 잘못된 방향으로 학습 및 발산해버릴 수 있기 때문이다.
        # 이때 그래디언트 클리핑gradient clipping이 도움이 된다. 
        # clip_grad_norm_()는 모든 기울기를 함께 스케일(scale) 하는 함수이다. 

        optimizer.step()
        # Optimizer의 step 함수를 호출하면 매개변수가 갱신된다.

        epoch_loss += loss.item()
        # loss.item()은 loss의 스칼라 값이다.
        # 전체 손실 값 계산

    return epoch_loss / len(iterator)

In [106]:
# 모델 평가(evaluate) 함수

def evaluate(model, iterator, criterion):
    model.eval() # 평가모드
    epoch_loss = 0

    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg

            output, _ = model(src, trg[:,:-1])
            output_dim = output.shape[-1]
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)

            loss = criterion(output, trg)
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)    
        

- 학습 및 검증 진행 
    * 학습 횟수(epoch) : 10

In [107]:
# 학습 경과 시간 확인 함수

import math
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time/60)
    elapsed_secs = int(elapsed_time - (elapsed_mins*60))
    return elapsed_mins, elapsed_secs

In [109]:
import time
import math
import random

N_EPOCHS = 10 # 10 에포크 학습
CLIP = 1
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time() #시작 시작 기록

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time() # 종료 시간 기록
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss  # 학습하면서 가장 작은 손실 평가를 갱신함
        torch.save(model.state_dict(), 'transformer_german_to_english.pt')
        # state_dict는 모델 parameters를 Tensor로 매핑한 Python dict 객체

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
                # 정수 2자리로 표현
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')
           # \t는 탭 들여쓰기             # 실수 소수점 3자리까지 표현
        #    Math.exp()함수는 x를 인수로 하는 e^x 값을 반환, 여기서는 e는 Euler 의 상수 e (약 2.71828), e^train_loss의 형태
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):.3f}')
    # 펄플렉서티(perplexity, PPL)는 언어 모델을 평가하기 위한 내부 평가(Intrinsic evaluation) 지표이다. '낮을수록' 언어 모델의 성능이 좋다는 것을 의미한다.
    # 언어 모델의 PPL이 10이 나왔다면, 해당 언어 모델은 테스트 데이터에 대해서 다음 단어를 예측하는 모든 시점(time-step)마다 
    # 평균적으로 10개의 단어를 가지고 어떤 것이 정답인지 고민하고 있다고 볼 수 있다.

Epoch: 01 | Time: 0m 18s
	Train Loss: 2.831 | Train PPL: 16.958
	Validation Loss: 2.324 | Validation PPL: 10.217
Epoch: 02 | Time: 0m 19s
	Train Loss: 2.253 | Train PPL: 9.514
	Validation Loss: 2.001 | Validation PPL: 7.399
Epoch: 03 | Time: 0m 19s
	Train Loss: 1.895 | Train PPL: 6.654
	Validation Loss: 1.805 | Validation PPL: 6.080
Epoch: 04 | Time: 0m 19s
	Train Loss: 1.646 | Train PPL: 5.188
	Validation Loss: 1.715 | Validation PPL: 5.555
Epoch: 05 | Time: 0m 19s
	Train Loss: 1.458 | Train PPL: 4.297
	Validation Loss: 1.661 | Validation PPL: 5.264
Epoch: 06 | Time: 0m 19s
	Train Loss: 1.306 | Train PPL: 3.691
	Validation Loss: 1.639 | Validation PPL: 5.149
Epoch: 07 | Time: 0m 19s
	Train Loss: 1.176 | Train PPL: 3.241
	Validation Loss: 1.628 | Validation PPL: 5.092
Epoch: 08 | Time: 0m 19s
	Train Loss: 1.067 | Train PPL: 2.906
	Validation Loss: 1.631 | Validation PPL: 5.111
Epoch: 09 | Time: 0m 19s
	Train Loss: 0.972 | Train PPL: 2.642
	Validation Loss: 1.656 | Validation PPL: 5.241

In [110]:
# 학습된 모델 저장
from google.colab import files

files.download('transformer_german_to_english.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

- 모델 테스트(testing) 진행

In [111]:
model.load_state_dict(torch.load('transformer_german_to_english.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):.3f}')

Test Loss: 1.664 | Test PPL: 5.278
